In [12]:
import os
import docx2txt
import pandas as pd
import pymysql
from pathlib import Path
# custom
from src.pdfPreprocessor import PdfDocument
from src.docPreprocessor import DocxDocument

In [2]:
path = Path(Path.cwd())
par = str(path.parent)
data_path = par + "/datafiles/raw-data/"

In [3]:
from datetime import datetime

today = datetime.today()

todayDate = str(today.year) + "-" + str(today.month) + "-" + \
            str(today.day) + "-" + str(today.hour) + "-"

In [10]:
# docx 파일 처리

f = open(todayDate + 'log.txt', 'wt')

letter_list = []

for datadir in os.listdir(data_path):
    if datadir != 'incruit' and datadir != 'incruit2':
        continue

    cnt = 0
    
    for file in os.listdir(data_path + datadir):
        cnt += 1
        ext = file[-3:]
        if ext == 'ocx':
            text = docx2txt.process(data_path + datadir + "/" + file)
            lines = []
            for line in text.splitlines():
                if line == '\n' or line == '':
                    continue
                lines.append(line)
            docObj = DocxDocument(lines)
            corpName = docObj.extractCorpName()
            task = docObj.extractTask()
            q, a = docObj.extractQAList()
            
            if len(q) != len(a):
                f.write(data_path + datadir + '/' + file + '\n')
                continue
                
            for i in range(len(q)):
                idx = a[i].find('\t')
                if idx != -1:
                    a[i] = a[i][:idx]
                letter_list.append([corpName, task, q[i][2:].strip(), a[i].strip()])
        
    print("디렉터리 {}의 {}개 파일을 처리 완료.".format(datadir, cnt))
    
f.close()

디렉터리 incruit2의 1412개 파일을 처리 완료.
디렉터리 incruit의 1887개 파일을 처리 완료.


## MySQL

In [13]:
conn_info = {'host':'13.124.236.54', \
           'user':'ssac', \
           'password':'Ssac123!@#', \
           'db':'job4',\
           'charset':'utf8'}

In [16]:
letter_list[-10]

['KB손해사정',
 '재물배상 파트',
 '희망업무 및 장래의 포부',
 '[배상책임보험의 손해사정 그리고 CPCU 취득을 통한 최고의 손해보험전문가]  배상책임보험의 손해사정 업무를 하고 싶습니다. 배상책임보험은 현재 가장 활성화되고 있는 손해보험종목 중 하나라고 생각합니다. 피해자보호정책이 강화되고 가해자의 배상이행자력을 확보하기 위해 배상책임보험은 더욱 성장 발전 할 것입니다. KB홈앤비즈케어종합보험에는 임차자 배상책임에서부터 학교경영자 배상책임까지 다양한 손해사정을 통해 개인과 기업 모두에게 만족을 드리는 서비스를 제공하고 싶습니다. 이름만으로도 신뢰감을 주는 회사이기 때문에 그 일원으로서 더욱 보람을 느끼며 일할 수 있다고 생각합니다. 고액의 보험금이 저의 판단으로 지급, 부 지급이 결정되기 때문에 더욱 더 책임감을 느끼고 업무에 일 할 수 있는 것이 재물배상 손해사정 직무의 장점이라고 생각합니다. 재물보험 전문가로서 더 성장하기 위해 국제적 손해보험전문인 자격증인 CPCU를 취득하여 고객들에게 더 깊은 신뢰를 드리겠습니다. 사고가 나면 보험금 이야기만 하는 딱딱한 직장인이 아닌 위로가 먼저 되는 사람이 되겠습니다. 제 자신이 먼저 자부심을 가지고 KB손해사정의 가치를 더욱 더 높이는 사람이 되겠습니다']

In [17]:
task = letter_list[-10][1]

In [25]:
conn = pymysql.connect( **conn_info )

cursor = conn.cursor()

sql = """select `task_id` from `myauth_task` where `name` like %s"""

cursor.execute(sql, ('%' + task + '%'))

result = cursor.fetchall()

task_id = -1

for row in result:
    task_id = row[0]
    
print(result)
        
conn.close()

1
((529,),)
